In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///dmeyf.db

###obviar estas celdas para jupyter
"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"



In [3]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto('competencia_03_NA.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


##Ver la tabla

In [4]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_03';

,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT
153,Visa_mpagominimo,DOUBLE


#Features históricas

In [5]:
# Selecciono las 30 variables de mayor importancia en el Random Forest de la competencia 1

campos = ['cdescubierto_preacordado','mcuenta_corriente', 'visa_status','ctrx_quarter', 'mpasivos_margen', 'mautoservicio' , 'ctarjeta_debito_transacciones' , 'mtransferencias_emitidas' , 'ctransferencias_emitidas' , 'mpayroll' , 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo','mprestamos_personales',
'mrentabilidad_annual', 'mrentabilidad', 'ctarjeta_visa_transacciones', 'mactivos_margen', 'cprestamos_personales', 'Visa_msaldopesos', 'Visa_msaldototal', 'Visa_mpagominimo', 'mextraccion_autoservicio', 'cproductos', 'matm' ]
print(campos)
print(len(campos))

['cdescubierto_preacordado', 'mcuenta_corriente', 'visa_status', 'ctrx_quarter', 'mpasivos_margen', 'mautoservicio', 'ctarjeta_debito_transacciones', 'mtransferencias_emitidas', 'ctransferencias_emitidas', 'mpayroll', 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo', 'mprestamos_personales', 'mrentabilidad_annual', 'mrentabilidad', 'ctarjeta_visa_transacciones', 'mactivos_margen', 'cprestamos_personales', 'Visa_msaldopesos', 'Visa_msaldototal', 'Visa_mpagominimo', 'mcuenta_corriente', 'mextraccion_autoservicio', 'cproductos', 'matm']
30


In [6]:
#lag, delta lag, slope, media, min y max para ventana de 5 meses
len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f", lag({campo}, 1) over ventana_5 as {campo}_lag_1"
  nuevos_features += f", lag({campo}, 2) over ventana_5 as {campo}_lag_2"
  nuevos_features += f", lag({campo}, 3) over ventana_5 as {campo}_lag_3"
  nuevos_features += f", lag({campo}, 4) over ventana_5 as {campo}_lag_4"
  nuevos_features += f", lag({campo}, 5) over ventana_5 as {campo}_lag_5"
  nuevos_features += f", {campo} - lag({campo}, 1) over ventana_5 as {campo}_delta_1_lag"
  nuevos_features += f", {campo} - lag({campo}, 2) over ventana_5 as {campo}_delta_2_lag"
  nuevos_features += f", {campo} - lag({campo}, 3) over ventana_5 as {campo}_delta_3_lag"
  nuevos_features += f", {campo} - lag({campo}, 4) over ventana_5 as {campo}_delta_4_lag"
  nuevos_features += f", {campo} - lag({campo}, 5) over ventana_5 as {campo}_delta_5_lag"
  nuevos_features += f", avg({campo}) over ventana_5 as {campo}_media_5"
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_5 as {campo}_slope_5"
  nuevos_features += f", max({campo}) over ventana_5 as {campo}_max_5"
  nuevos_features += f", min({campo}) over ventana_5 as {campo}_min_5"

print(nuevos_features)




, lag(cdescubierto_preacordado, 1) over ventana_5 as cdescubierto_preacordado_lag_1, lag(cdescubierto_preacordado, 2) over ventana_5 as cdescubierto_preacordado_lag_2, lag(cdescubierto_preacordado, 3) over ventana_5 as cdescubierto_preacordado_lag_3, lag(cdescubierto_preacordado, 4) over ventana_5 as cdescubierto_preacordado_lag_4, lag(cdescubierto_preacordado, 5) over ventana_5 as cdescubierto_preacordado_lag_5, cdescubierto_preacordado - lag(cdescubierto_preacordado, 1) over ventana_5 as cdescubierto_preacordado_delta_1_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 2) over ventana_5 as cdescubierto_preacordado_delta_2_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 3) over ventana_5 as cdescubierto_preacordado_delta_3_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 4) over ventana_5 as cdescubierto_preacordado_delta_4_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 5) over ventana_5 as cdescubierto_preacordado_delta_5_lag, avg

In [7]:
%%time
%%sql
create or replace table competencia_03 as
SELECT
  *
, lag(cdescubierto_preacordado, 1) over ventana_5 as cdescubierto_preacordado_lag_1, lag(cdescubierto_preacordado, 2) over ventana_5 as cdescubierto_preacordado_lag_2, lag(cdescubierto_preacordado, 3) over ventana_5 as cdescubierto_preacordado_lag_3, lag(cdescubierto_preacordado, 4) over ventana_5 as cdescubierto_preacordado_lag_4, lag(cdescubierto_preacordado, 5) over ventana_5 as cdescubierto_preacordado_lag_5, cdescubierto_preacordado - lag(cdescubierto_preacordado, 1) over ventana_5 as cdescubierto_preacordado_delta_1_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 2) over ventana_5 as cdescubierto_preacordado_delta_2_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 3) over ventana_5 as cdescubierto_preacordado_delta_3_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 4) over ventana_5 as cdescubierto_preacordado_delta_4_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 5) over ventana_5 as cdescubierto_preacordado_delta_5_lag, avg(cdescubierto_preacordado) over ventana_5 as cdescubierto_preacordado_media_5
, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_5 as cdescubierto_preacordado_slope_5, max(cdescubierto_preacordado) over ventana_5 as cdescubierto_preacordado_max_5, min(cdescubierto_preacordado) over ventana_5 as cdescubierto_preacordado_min_5, lag(mcuenta_corriente, 1) over ventana_5 as mcuenta_corriente_lag_1, lag(mcuenta_corriente, 2) over ventana_5 as mcuenta_corriente_lag_2, lag(mcuenta_corriente, 3) over ventana_5 as mcuenta_corriente_lag_3, lag(mcuenta_corriente, 4) over ventana_5 as mcuenta_corriente_lag_4, lag(mcuenta_corriente, 5) over ventana_5 as mcuenta_corriente_lag_5, mcuenta_corriente - lag(mcuenta_corriente, 1) over ventana_5 as mcuenta_corriente_delta_1_lag, mcuenta_corriente - lag(mcuenta_corriente, 2) over ventana_5 as mcuenta_corriente_delta_2_lag, mcuenta_corriente - lag(mcuenta_corriente, 3) over ventana_5 as mcuenta_corriente_delta_3_lag, mcuenta_corriente - lag(mcuenta_corriente, 4) over ventana_5 as mcuenta_corriente_delta_4_lag, mcuenta_corriente - lag(mcuenta_corriente, 5) over ventana_5 as mcuenta_corriente_delta_5_lag, avg(mcuenta_corriente) over ventana_5 as mcuenta_corriente_media_5
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_5 as mcuenta_corriente_slope_5, max(mcuenta_corriente) over ventana_5 as mcuenta_corriente_max_5, min(mcuenta_corriente) over ventana_5 as mcuenta_corriente_min_5, lag(visa_status, 1) over ventana_5 as visa_status_lag_1, lag(visa_status, 2) over ventana_5 as visa_status_lag_2, lag(visa_status, 3) over ventana_5 as visa_status_lag_3, lag(visa_status, 4) over ventana_5 as visa_status_lag_4, lag(visa_status, 5) over ventana_5 as visa_status_lag_5, visa_status - lag(visa_status, 1) over ventana_5 as visa_status_delta_1_lag, visa_status - lag(visa_status, 2) over ventana_5 as visa_status_delta_2_lag, visa_status - lag(visa_status, 3) over ventana_5 as visa_status_delta_3_lag, visa_status - lag(visa_status, 4) over ventana_5 as visa_status_delta_4_lag, visa_status - lag(visa_status, 5) over ventana_5 as visa_status_delta_5_lag, avg(visa_status) over ventana_5 as visa_status_media_5
, regr_slope(visa_status, cliente_antiguedad) over ventana_5 as visa_status_slope_5, max(visa_status) over ventana_5 as visa_status_max_5, min(visa_status) over ventana_5 as visa_status_min_5, lag(ctrx_quarter, 1) over ventana_5 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_5 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_5 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_5 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_5 as ctrx_quarter_lag_5, ctrx_quarter - lag(ctrx_quarter, 1) over ventana_5 as ctrx_quarter_delta_1_lag, ctrx_quarter - lag(ctrx_quarter, 2) over ventana_5 as ctrx_quarter_delta_2_lag, ctrx_quarter - lag(ctrx_quarter, 3) over ventana_5 as ctrx_quarter_delta_3_lag, ctrx_quarter - lag(ctrx_quarter, 4) over ventana_5 as ctrx_quarter_delta_4_lag, ctrx_quarter - lag(ctrx_quarter, 5) over ventana_5 as ctrx_quarter_delta_5_lag, avg(ctrx_quarter) over ventana_5 as ctrx_quarter_media_5
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_5 as ctrx_quarter_slope_5, max(ctrx_quarter) over ventana_5 as ctrx_quarter_max_5, min(ctrx_quarter) over ventana_5 as ctrx_quarter_min_5, lag(mpasivos_margen, 1) over ventana_5 as mpasivos_margen_lag_1, lag(mpasivos_margen, 2) over ventana_5 as mpasivos_margen_lag_2, lag(mpasivos_margen, 3) over ventana_5 as mpasivos_margen_lag_3, lag(mpasivos_margen, 4) over ventana_5 as mpasivos_margen_lag_4, lag(mpasivos_margen, 5) over ventana_5 as mpasivos_margen_lag_5, mpasivos_margen - lag(mpasivos_margen, 1) over ventana_5 as mpasivos_margen_delta_1_lag, mpasivos_margen - lag(mpasivos_margen, 2) over ventana_5 as mpasivos_margen_delta_2_lag, mpasivos_margen - lag(mpasivos_margen, 3) over ventana_5 as mpasivos_margen_delta_3_lag, mpasivos_margen - lag(mpasivos_margen, 4) over ventana_5 as mpasivos_margen_delta_4_lag, mpasivos_margen - lag(mpasivos_margen, 5) over ventana_5 as mpasivos_margen_delta_5_lag, avg(mpasivos_margen) over ventana_5 as mpasivos_margen_media_5
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_5 as mpasivos_margen_slope_5, max(mpasivos_margen) over ventana_5 as mpasivos_margen_max_5, min(mpasivos_margen) over ventana_5 as mpasivos_margen_min_5, lag(mautoservicio, 1) over ventana_5 as mautoservicio_lag_1, lag(mautoservicio, 2) over ventana_5 as mautoservicio_lag_2, lag(mautoservicio, 3) over ventana_5 as mautoservicio_lag_3, lag(mautoservicio, 4) over ventana_5 as mautoservicio_lag_4, lag(mautoservicio, 5) over ventana_5 as mautoservicio_lag_5, mautoservicio - lag(mautoservicio, 1) over ventana_5 as mautoservicio_delta_1_lag, mautoservicio - lag(mautoservicio, 2) over ventana_5 as mautoservicio_delta_2_lag, mautoservicio - lag(mautoservicio, 3) over ventana_5 as mautoservicio_delta_3_lag, mautoservicio - lag(mautoservicio, 4) over ventana_5 as mautoservicio_delta_4_lag, mautoservicio - lag(mautoservicio, 5) over ventana_5 as mautoservicio_delta_5_lag, avg(mautoservicio) over ventana_5 as mautoservicio_media_5
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_5 as mautoservicio_slope_5, max(mautoservicio) over ventana_5 as mautoservicio_max_5, min(mautoservicio) over ventana_5 as mautoservicio_min_5, lag(ctarjeta_debito_transacciones, 1) over ventana_5 as ctarjeta_debito_transacciones_lag_1, lag(ctarjeta_debito_transacciones, 2) over ventana_5 as ctarjeta_debito_transacciones_lag_2, lag(ctarjeta_debito_transacciones, 3) over ventana_5 as ctarjeta_debito_transacciones_lag_3, lag(ctarjeta_debito_transacciones, 4) over ventana_5 as ctarjeta_debito_transacciones_lag_4, lag(ctarjeta_debito_transacciones, 5) over ventana_5 as ctarjeta_debito_transacciones_lag_5, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 1) over ventana_5 as ctarjeta_debito_transacciones_delta_1_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 2) over ventana_5 as ctarjeta_debito_transacciones_delta_2_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 3) over ventana_5 as ctarjeta_debito_transacciones_delta_3_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 4) over ventana_5 as ctarjeta_debito_transacciones_delta_4_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 5) over ventana_5 as ctarjeta_debito_transacciones_delta_5_lag, avg(ctarjeta_debito_transacciones) over ventana_5 as ctarjeta_debito_transacciones_media_5
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_5 as ctarjeta_debito_transacciones_slope_5, max(ctarjeta_debito_transacciones) over ventana_5 as ctarjeta_debito_transacciones_max_5, min(ctarjeta_debito_transacciones) over ventana_5 as ctarjeta_debito_transacciones_min_5, lag(mtransferencias_emitidas, 1) over ventana_5 as mtransferencias_emitidas_lag_1, lag(mtransferencias_emitidas, 2) over ventana_5 as mtransferencias_emitidas_lag_2, lag(mtransferencias_emitidas, 3) over ventana_5 as mtransferencias_emitidas_lag_3, lag(mtransferencias_emitidas, 4) over ventana_5 as mtransferencias_emitidas_lag_4, lag(mtransferencias_emitidas, 5) over ventana_5 as mtransferencias_emitidas_lag_5, mtransferencias_emitidas - lag(mtransferencias_emitidas, 1) over ventana_5 as mtransferencias_emitidas_delta_1_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 2) over ventana_5 as mtransferencias_emitidas_delta_2_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 3) over ventana_5 as mtransferencias_emitidas_delta_3_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 4) over ventana_5 as mtransferencias_emitidas_delta_4_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 5) over ventana_5 as mtransferencias_emitidas_delta_5_lag, avg(mtransferencias_emitidas) over ventana_5 as mtransferencias_emitidas_media_5
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_5 as mtransferencias_emitidas_slope_5, max(mtransferencias_emitidas) over ventana_5 as mtransferencias_emitidas_max_5, min(mtransferencias_emitidas) over ventana_5 as mtransferencias_emitidas_min_5, lag(ctransferencias_emitidas, 1) over ventana_5 as ctransferencias_emitidas_lag_1, lag(ctransferencias_emitidas, 2) over ventana_5 as ctransferencias_emitidas_lag_2, lag(ctransferencias_emitidas, 3) over ventana_5 as ctransferencias_emitidas_lag_3, lag(ctransferencias_emitidas, 4) over ventana_5 as ctransferencias_emitidas_lag_4, lag(ctransferencias_emitidas, 5) over ventana_5 as ctransferencias_emitidas_lag_5, ctransferencias_emitidas - lag(ctransferencias_emitidas, 1) over ventana_5 as ctransferencias_emitidas_delta_1_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 2) over ventana_5 as ctransferencias_emitidas_delta_2_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 3) over ventana_5 as ctransferencias_emitidas_delta_3_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 4) over ventana_5 as ctransferencias_emitidas_delta_4_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 5) over ventana_5 as ctransferencias_emitidas_delta_5_lag, avg(ctransferencias_emitidas) over ventana_5 as ctransferencias_emitidas_media_5
, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_5 as ctransferencias_emitidas_slope_5, max(ctransferencias_emitidas) over ventana_5 as ctransferencias_emitidas_max_5, min(ctransferencias_emitidas) over ventana_5 as ctransferencias_emitidas_min_5, lag(mpayroll, 1) over ventana_5 as mpayroll_lag_1, lag(mpayroll, 2) over ventana_5 as mpayroll_lag_2, lag(mpayroll, 3) over ventana_5 as mpayroll_lag_3, lag(mpayroll, 4) over ventana_5 as mpayroll_lag_4, lag(mpayroll, 5) over ventana_5 as mpayroll_lag_5, mpayroll - lag(mpayroll, 1) over ventana_5 as mpayroll_delta_1_lag, mpayroll - lag(mpayroll, 2) over ventana_5 as mpayroll_delta_2_lag, mpayroll - lag(mpayroll, 3) over ventana_5 as mpayroll_delta_3_lag, mpayroll - lag(mpayroll, 4) over ventana_5 as mpayroll_delta_4_lag, mpayroll - lag(mpayroll, 5) over ventana_5 as mpayroll_delta_5_lag, avg(mpayroll) over ventana_5 as mpayroll_media_5
, regr_slope(mpayroll, cliente_antiguedad) over ventana_5 as mpayroll_slope_5, max(mpayroll) over ventana_5 as mpayroll_max_5, min(mpayroll) over ventana_5 as mpayroll_min_5, lag(cpayroll_trx, 1) over ventana_5 as cpayroll_trx_lag_1, lag(cpayroll_trx, 2) over ventana_5 as cpayroll_trx_lag_2, lag(cpayroll_trx, 3) over ventana_5 as cpayroll_trx_lag_3, lag(cpayroll_trx, 4) over ventana_5 as cpayroll_trx_lag_4, lag(cpayroll_trx, 5) over ventana_5 as cpayroll_trx_lag_5, cpayroll_trx - lag(cpayroll_trx, 1) over ventana_5 as cpayroll_trx_delta_1_lag, cpayroll_trx - lag(cpayroll_trx, 2) over ventana_5 as cpayroll_trx_delta_2_lag, cpayroll_trx - lag(cpayroll_trx, 3) over ventana_5 as cpayroll_trx_delta_3_lag, cpayroll_trx - lag(cpayroll_trx, 4) over ventana_5 as cpayroll_trx_delta_4_lag, cpayroll_trx - lag(cpayroll_trx, 5) over ventana_5 as cpayroll_trx_delta_5_lag, avg(cpayroll_trx) over ventana_5 as cpayroll_trx_media_5
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_5 as cpayroll_trx_slope_5, max(cpayroll_trx) over ventana_5 as cpayroll_trx_max_5, min(cpayroll_trx) over ventana_5 as cpayroll_trx_min_5, lag(mcomisiones_mantenimiento, 1) over ventana_5 as mcomisiones_mantenimiento_lag_1, lag(mcomisiones_mantenimiento, 2) over ventana_5 as mcomisiones_mantenimiento_lag_2, lag(mcomisiones_mantenimiento, 3) over ventana_5 as mcomisiones_mantenimiento_lag_3, lag(mcomisiones_mantenimiento, 4) over ventana_5 as mcomisiones_mantenimiento_lag_4, lag(mcomisiones_mantenimiento, 5) over ventana_5 as mcomisiones_mantenimiento_lag_5, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 1) over ventana_5 as mcomisiones_mantenimiento_delta_1_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 2) over ventana_5 as mcomisiones_mantenimiento_delta_2_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 3) over ventana_5 as mcomisiones_mantenimiento_delta_3_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 4) over ventana_5 as mcomisiones_mantenimiento_delta_4_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 5) over ventana_5 as mcomisiones_mantenimiento_delta_5_lag, avg(mcomisiones_mantenimiento) over ventana_5 as mcomisiones_mantenimiento_media_5
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_5 as mcomisiones_mantenimiento_slope_5, max(mcomisiones_mantenimiento) over ventana_5 as mcomisiones_mantenimiento_max_5, min(mcomisiones_mantenimiento) over ventana_5 as mcomisiones_mantenimiento_min_5, lag(mcomisiones_otras, 1) over ventana_5 as mcomisiones_otras_lag_1, lag(mcomisiones_otras, 2) over ventana_5 as mcomisiones_otras_lag_2, lag(mcomisiones_otras, 3) over ventana_5 as mcomisiones_otras_lag_3, lag(mcomisiones_otras, 4) over ventana_5 as mcomisiones_otras_lag_4, lag(mcomisiones_otras, 5) over ventana_5 as mcomisiones_otras_lag_5, mcomisiones_otras - lag(mcomisiones_otras, 1) over ventana_5 as mcomisiones_otras_delta_1_lag, mcomisiones_otras - lag(mcomisiones_otras, 2) over ventana_5 as mcomisiones_otras_delta_2_lag, mcomisiones_otras - lag(mcomisiones_otras, 3) over ventana_5 as mcomisiones_otras_delta_3_lag, mcomisiones_otras - lag(mcomisiones_otras, 4) over ventana_5 as mcomisiones_otras_delta_4_lag, mcomisiones_otras - lag(mcomisiones_otras, 5) over ventana_5 as mcomisiones_otras_delta_5_lag, avg(mcomisiones_otras) over ventana_5 as mcomisiones_otras_media_5
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_5 as mcomisiones_otras_slope_5, max(mcomisiones_otras) over ventana_5 as mcomisiones_otras_max_5, min(mcomisiones_otras) over ventana_5 as mcomisiones_otras_min_5, lag(mcomisiones, 1) over ventana_5 as mcomisiones_lag_1, lag(mcomisiones, 2) over ventana_5 as mcomisiones_lag_2, lag(mcomisiones, 3) over ventana_5 as mcomisiones_lag_3, lag(mcomisiones, 4) over ventana_5 as mcomisiones_lag_4, lag(mcomisiones, 5) over ventana_5 as mcomisiones_lag_5, mcomisiones - lag(mcomisiones, 1) over ventana_5 as mcomisiones_delta_1_lag, mcomisiones - lag(mcomisiones, 2) over ventana_5 as mcomisiones_delta_2_lag, mcomisiones - lag(mcomisiones, 3) over ventana_5 as mcomisiones_delta_3_lag, mcomisiones - lag(mcomisiones, 4) over ventana_5 as mcomisiones_delta_4_lag, mcomisiones - lag(mcomisiones, 5) over ventana_5 as mcomisiones_delta_5_lag, avg(mcomisiones) over ventana_5 as mcomisiones_media_5
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_5 as mcomisiones_slope_5, max(mcomisiones) over ventana_5 as mcomisiones_max_5, min(mcomisiones) over ventana_5 as mcomisiones_min_5, lag(mcaja_ahorro, 1) over ventana_5 as mcaja_ahorro_lag_1, lag(mcaja_ahorro, 2) over ventana_5 as mcaja_ahorro_lag_2, lag(mcaja_ahorro, 3) over ventana_5 as mcaja_ahorro_lag_3, lag(mcaja_ahorro, 4) over ventana_5 as mcaja_ahorro_lag_4, lag(mcaja_ahorro, 5) over ventana_5 as mcaja_ahorro_lag_5, mcaja_ahorro - lag(mcaja_ahorro, 1) over ventana_5 as mcaja_ahorro_delta_1_lag, mcaja_ahorro - lag(mcaja_ahorro, 2) over ventana_5 as mcaja_ahorro_delta_2_lag, mcaja_ahorro - lag(mcaja_ahorro, 3) over ventana_5 as mcaja_ahorro_delta_3_lag, mcaja_ahorro - lag(mcaja_ahorro, 4) over ventana_5 as mcaja_ahorro_delta_4_lag, mcaja_ahorro - lag(mcaja_ahorro, 5) over ventana_5 as mcaja_ahorro_delta_5_lag, avg(mcaja_ahorro) over ventana_5 as mcaja_ahorro_media_5
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_5 as mcaja_ahorro_slope_5, max(mcaja_ahorro) over ventana_5 as mcaja_ahorro_max_5, min(mcaja_ahorro) over ventana_5 as mcaja_ahorro_min_5, lag(mcuentas_saldo, 1) over ventana_5 as mcuentas_saldo_lag_1, lag(mcuentas_saldo, 2) over ventana_5 as mcuentas_saldo_lag_2, lag(mcuentas_saldo, 3) over ventana_5 as mcuentas_saldo_lag_3, lag(mcuentas_saldo, 4) over ventana_5 as mcuentas_saldo_lag_4, lag(mcuentas_saldo, 5) over ventana_5 as mcuentas_saldo_lag_5, mcuentas_saldo - lag(mcuentas_saldo, 1) over ventana_5 as mcuentas_saldo_delta_1_lag, mcuentas_saldo - lag(mcuentas_saldo, 2) over ventana_5 as mcuentas_saldo_delta_2_lag, mcuentas_saldo - lag(mcuentas_saldo, 3) over ventana_5 as mcuentas_saldo_delta_3_lag, mcuentas_saldo - lag(mcuentas_saldo, 4) over ventana_5 as mcuentas_saldo_delta_4_lag, mcuentas_saldo - lag(mcuentas_saldo, 5) over ventana_5 as mcuentas_saldo_delta_5_lag, avg(mcuentas_saldo) over ventana_5 as mcuentas_saldo_media_5
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_5 as mcuentas_saldo_slope_5, max(mcuentas_saldo) over ventana_5 as mcuentas_saldo_max_5, min(mcuentas_saldo) over ventana_5 as mcuentas_saldo_min_5, lag(mtarjeta_visa_consumo, 1) over ventana_5 as mtarjeta_visa_consumo_lag_1, lag(mtarjeta_visa_consumo, 2) over ventana_5 as mtarjeta_visa_consumo_lag_2, lag(mtarjeta_visa_consumo, 3) over ventana_5 as mtarjeta_visa_consumo_lag_3, lag(mtarjeta_visa_consumo, 4) over ventana_5 as mtarjeta_visa_consumo_lag_4, lag(mtarjeta_visa_consumo, 5) over ventana_5 as mtarjeta_visa_consumo_lag_5, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 1) over ventana_5 as mtarjeta_visa_consumo_delta_1_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 2) over ventana_5 as mtarjeta_visa_consumo_delta_2_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 3) over ventana_5 as mtarjeta_visa_consumo_delta_3_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 4) over ventana_5 as mtarjeta_visa_consumo_delta_4_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 5) over ventana_5 as mtarjeta_visa_consumo_delta_5_lag, avg(mtarjeta_visa_consumo) over ventana_5 as mtarjeta_visa_consumo_media_5
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_5 as mtarjeta_visa_consumo_slope_5, max(mtarjeta_visa_consumo) over ventana_5 as mtarjeta_visa_consumo_max_5, min(mtarjeta_visa_consumo) over ventana_5 as mtarjeta_visa_consumo_min_5, lag(mprestamos_personales, 1) over ventana_5 as mprestamos_personales_lag_1, lag(mprestamos_personales, 2) over ventana_5 as mprestamos_personales_lag_2, lag(mprestamos_personales, 3) over ventana_5 as mprestamos_personales_lag_3, lag(mprestamos_personales, 4) over ventana_5 as mprestamos_personales_lag_4, lag(mprestamos_personales, 5) over ventana_5 as mprestamos_personales_lag_5, mprestamos_personales - lag(mprestamos_personales, 1) over ventana_5 as mprestamos_personales_delta_1_lag, mprestamos_personales - lag(mprestamos_personales, 2) over ventana_5 as mprestamos_personales_delta_2_lag, mprestamos_personales - lag(mprestamos_personales, 3) over ventana_5 as mprestamos_personales_delta_3_lag, mprestamos_personales - lag(mprestamos_personales, 4) over ventana_5 as mprestamos_personales_delta_4_lag, mprestamos_personales - lag(mprestamos_personales, 5) over ventana_5 as mprestamos_personales_delta_5_lag, avg(mprestamos_personales) over ventana_5 as mprestamos_personales_media_5
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_5 as mprestamos_personales_slope_5, max(mprestamos_personales) over ventana_5 as mprestamos_personales_max_5, min(mprestamos_personales) over ventana_5 as mprestamos_personales_min_5, lag(mrentabilidad_annual, 1) over ventana_5 as mrentabilidad_annual_lag_1, lag(mrentabilidad_annual, 2) over ventana_5 as mrentabilidad_annual_lag_2, lag(mrentabilidad_annual, 3) over ventana_5 as mrentabilidad_annual_lag_3, lag(mrentabilidad_annual, 4) over ventana_5 as mrentabilidad_annual_lag_4, lag(mrentabilidad_annual, 5) over ventana_5 as mrentabilidad_annual_lag_5, mrentabilidad_annual - lag(mrentabilidad_annual, 1) over ventana_5 as mrentabilidad_annual_delta_1_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 2) over ventana_5 as mrentabilidad_annual_delta_2_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 3) over ventana_5 as mrentabilidad_annual_delta_3_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 4) over ventana_5 as mrentabilidad_annual_delta_4_lag, mrentabilidad_annual - lag(mrentabilidad_annual, 5) over ventana_5 as mrentabilidad_annual_delta_5_lag, avg(mrentabilidad_annual) over ventana_5 as mrentabilidad_annual_media_5
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_5 as mrentabilidad_annual_slope_5, max(mrentabilidad_annual) over ventana_5 as mrentabilidad_annual_max_5, min(mrentabilidad_annual) over ventana_5 as mrentabilidad_annual_min_5, lag(mrentabilidad, 1) over ventana_5 as mrentabilidad_lag_1, lag(mrentabilidad, 2) over ventana_5 as mrentabilidad_lag_2, lag(mrentabilidad, 3) over ventana_5 as mrentabilidad_lag_3, lag(mrentabilidad, 4) over ventana_5 as mrentabilidad_lag_4, lag(mrentabilidad, 5) over ventana_5 as mrentabilidad_lag_5, mrentabilidad - lag(mrentabilidad, 1) over ventana_5 as mrentabilidad_delta_1_lag, mrentabilidad - lag(mrentabilidad, 2) over ventana_5 as mrentabilidad_delta_2_lag, mrentabilidad - lag(mrentabilidad, 3) over ventana_5 as mrentabilidad_delta_3_lag, mrentabilidad - lag(mrentabilidad, 4) over ventana_5 as mrentabilidad_delta_4_lag, mrentabilidad - lag(mrentabilidad, 5) over ventana_5 as mrentabilidad_delta_5_lag, avg(mrentabilidad) over ventana_5 as mrentabilidad_media_5
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_5 as mrentabilidad_slope_5, max(mrentabilidad) over ventana_5 as mrentabilidad_max_5, min(mrentabilidad) over ventana_5 as mrentabilidad_min_5, lag(ctarjeta_visa_transacciones, 1) over ventana_5 as ctarjeta_visa_transacciones_lag_1, lag(ctarjeta_visa_transacciones, 2) over ventana_5 as ctarjeta_visa_transacciones_lag_2, lag(ctarjeta_visa_transacciones, 3) over ventana_5 as ctarjeta_visa_transacciones_lag_3, lag(ctarjeta_visa_transacciones, 4) over ventana_5 as ctarjeta_visa_transacciones_lag_4, lag(ctarjeta_visa_transacciones, 5) over ventana_5 as ctarjeta_visa_transacciones_lag_5, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 1) over ventana_5 as ctarjeta_visa_transacciones_delta_1_lag, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 2) over ventana_5 as ctarjeta_visa_transacciones_delta_2_lag, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 3) over ventana_5 as ctarjeta_visa_transacciones_delta_3_lag, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 4) over ventana_5 as ctarjeta_visa_transacciones_delta_4_lag, ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 5) over ventana_5 as ctarjeta_visa_transacciones_delta_5_lag, avg(ctarjeta_visa_transacciones) over ventana_5 as ctarjeta_visa_transacciones_media_5
, regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_5 as ctarjeta_visa_transacciones_slope_5, max(ctarjeta_visa_transacciones) over ventana_5 as ctarjeta_visa_transacciones_max_5, min(ctarjeta_visa_transacciones) over ventana_5 as ctarjeta_visa_transacciones_min_5, lag(mactivos_margen, 1) over ventana_5 as mactivos_margen_lag_1, lag(mactivos_margen, 2) over ventana_5 as mactivos_margen_lag_2, lag(mactivos_margen, 3) over ventana_5 as mactivos_margen_lag_3, lag(mactivos_margen, 4) over ventana_5 as mactivos_margen_lag_4, lag(mactivos_margen, 5) over ventana_5 as mactivos_margen_lag_5, mactivos_margen - lag(mactivos_margen, 1) over ventana_5 as mactivos_margen_delta_1_lag, mactivos_margen - lag(mactivos_margen, 2) over ventana_5 as mactivos_margen_delta_2_lag, mactivos_margen - lag(mactivos_margen, 3) over ventana_5 as mactivos_margen_delta_3_lag, mactivos_margen - lag(mactivos_margen, 4) over ventana_5 as mactivos_margen_delta_4_lag, mactivos_margen - lag(mactivos_margen, 5) over ventana_5 as mactivos_margen_delta_5_lag, avg(mactivos_margen) over ventana_5 as mactivos_margen_media_5
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_5 as mactivos_margen_slope_5, max(mactivos_margen) over ventana_5 as mactivos_margen_max_5, min(mactivos_margen) over ventana_5 as mactivos_margen_min_5, lag(cprestamos_personales, 1) over ventana_5 as cprestamos_personales_lag_1, lag(cprestamos_personales, 2) over ventana_5 as cprestamos_personales_lag_2, lag(cprestamos_personales, 3) over ventana_5 as cprestamos_personales_lag_3, lag(cprestamos_personales, 4) over ventana_5 as cprestamos_personales_lag_4, lag(cprestamos_personales, 5) over ventana_5 as cprestamos_personales_lag_5, cprestamos_personales - lag(cprestamos_personales, 1) over ventana_5 as cprestamos_personales_delta_1_lag, cprestamos_personales - lag(cprestamos_personales, 2) over ventana_5 as cprestamos_personales_delta_2_lag, cprestamos_personales - lag(cprestamos_personales, 3) over ventana_5 as cprestamos_personales_delta_3_lag, cprestamos_personales - lag(cprestamos_personales, 4) over ventana_5 as cprestamos_personales_delta_4_lag, cprestamos_personales - lag(cprestamos_personales, 5) over ventana_5 as cprestamos_personales_delta_5_lag, avg(cprestamos_personales) over ventana_5 as cprestamos_personales_media_5
, regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_5 as cprestamos_personales_slope_5, max(cprestamos_personales) over ventana_5 as cprestamos_personales_max_5, min(cprestamos_personales) over ventana_5 as cprestamos_personales_min_5, lag(Visa_msaldopesos, 1) over ventana_5 as Visa_msaldopesos_lag_1, lag(Visa_msaldopesos, 2) over ventana_5 as Visa_msaldopesos_lag_2, lag(Visa_msaldopesos, 3) over ventana_5 as Visa_msaldopesos_lag_3, lag(Visa_msaldopesos, 4) over ventana_5 as Visa_msaldopesos_lag_4, lag(Visa_msaldopesos, 5) over ventana_5 as Visa_msaldopesos_lag_5, Visa_msaldopesos - lag(Visa_msaldopesos, 1) over ventana_5 as Visa_msaldopesos_delta_1_lag, Visa_msaldopesos - lag(Visa_msaldopesos, 2) over ventana_5 as Visa_msaldopesos_delta_2_lag, Visa_msaldopesos - lag(Visa_msaldopesos, 3) over ventana_5 as Visa_msaldopesos_delta_3_lag, Visa_msaldopesos - lag(Visa_msaldopesos, 4) over ventana_5 as Visa_msaldopesos_delta_4_lag, Visa_msaldopesos - lag(Visa_msaldopesos, 5) over ventana_5 as Visa_msaldopesos_delta_5_lag, avg(Visa_msaldopesos) over ventana_5 as Visa_msaldopesos_media_5
, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_5 as Visa_msaldopesos_slope_5, max(Visa_msaldopesos) over ventana_5 as Visa_msaldopesos_max_5, min(Visa_msaldopesos) over ventana_5 as Visa_msaldopesos_min_5, lag(Visa_msaldototal, 1) over ventana_5 as Visa_msaldototal_lag_1, lag(Visa_msaldototal, 2) over ventana_5 as Visa_msaldototal_lag_2, lag(Visa_msaldototal, 3) over ventana_5 as Visa_msaldototal_lag_3, lag(Visa_msaldototal, 4) over ventana_5 as Visa_msaldototal_lag_4, lag(Visa_msaldototal, 5) over ventana_5 as Visa_msaldototal_lag_5, Visa_msaldototal - lag(Visa_msaldototal, 1) over ventana_5 as Visa_msaldototal_delta_1_lag, Visa_msaldototal - lag(Visa_msaldototal, 2) over ventana_5 as Visa_msaldototal_delta_2_lag, Visa_msaldototal - lag(Visa_msaldototal, 3) over ventana_5 as Visa_msaldototal_delta_3_lag, Visa_msaldototal - lag(Visa_msaldototal, 4) over ventana_5 as Visa_msaldototal_delta_4_lag, Visa_msaldototal - lag(Visa_msaldototal, 5) over ventana_5 as Visa_msaldototal_delta_5_lag, avg(Visa_msaldototal) over ventana_5 as Visa_msaldototal_media_5
, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_5 as Visa_msaldototal_slope_5, max(Visa_msaldototal) over ventana_5 as Visa_msaldototal_max_5, min(Visa_msaldototal) over ventana_5 as Visa_msaldototal_min_5, lag(Visa_mpagominimo, 1) over ventana_5 as Visa_mpagominimo_lag_1, lag(Visa_mpagominimo, 2) over ventana_5 as Visa_mpagominimo_lag_2, lag(Visa_mpagominimo, 3) over ventana_5 as Visa_mpagominimo_lag_3, lag(Visa_mpagominimo, 4) over ventana_5 as Visa_mpagominimo_lag_4, lag(Visa_mpagominimo, 5) over ventana_5 as Visa_mpagominimo_lag_5, Visa_mpagominimo - lag(Visa_mpagominimo, 1) over ventana_5 as Visa_mpagominimo_delta_1_lag, Visa_mpagominimo - lag(Visa_mpagominimo, 2) over ventana_5 as Visa_mpagominimo_delta_2_lag, Visa_mpagominimo - lag(Visa_mpagominimo, 3) over ventana_5 as Visa_mpagominimo_delta_3_lag, Visa_mpagominimo - lag(Visa_mpagominimo, 4) over ventana_5 as Visa_mpagominimo_delta_4_lag, Visa_mpagominimo - lag(Visa_mpagominimo, 5) over ventana_5 as Visa_mpagominimo_delta_5_lag, avg(Visa_mpagominimo) over ventana_5 as Visa_mpagominimo_media_5
, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_5 as Visa_mpagominimo_slope_5, max(Visa_mpagominimo) over ventana_5 as Visa_mpagominimo_max_5, min(Visa_mpagominimo) over ventana_5 as Visa_mpagominimo_min_5, lag(mcuenta_corriente, 1) over ventana_5 as mcuenta_corriente_lag_1, lag(mcuenta_corriente, 2) over ventana_5 as mcuenta_corriente_lag_2, lag(mcuenta_corriente, 3) over ventana_5 as mcuenta_corriente_lag_3, lag(mcuenta_corriente, 4) over ventana_5 as mcuenta_corriente_lag_4, lag(mcuenta_corriente, 5) over ventana_5 as mcuenta_corriente_lag_5, mcuenta_corriente - lag(mcuenta_corriente, 1) over ventana_5 as mcuenta_corriente_delta_1_lag, mcuenta_corriente - lag(mcuenta_corriente, 2) over ventana_5 as mcuenta_corriente_delta_2_lag, mcuenta_corriente - lag(mcuenta_corriente, 3) over ventana_5 as mcuenta_corriente_delta_3_lag, mcuenta_corriente - lag(mcuenta_corriente, 4) over ventana_5 as mcuenta_corriente_delta_4_lag, mcuenta_corriente - lag(mcuenta_corriente, 5) over ventana_5 as mcuenta_corriente_delta_5_lag, avg(mcuenta_corriente) over ventana_5 as mcuenta_corriente_media_5
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_5 as mcuenta_corriente_slope_5, max(mcuenta_corriente) over ventana_5 as mcuenta_corriente_max_5, min(mcuenta_corriente) over ventana_5 as mcuenta_corriente_min_5, lag(mextraccion_autoservicio, 1) over ventana_5 as mextraccion_autoservicio_lag_1, lag(mextraccion_autoservicio, 2) over ventana_5 as mextraccion_autoservicio_lag_2, lag(mextraccion_autoservicio, 3) over ventana_5 as mextraccion_autoservicio_lag_3, lag(mextraccion_autoservicio, 4) over ventana_5 as mextraccion_autoservicio_lag_4, lag(mextraccion_autoservicio, 5) over ventana_5 as mextraccion_autoservicio_lag_5, mextraccion_autoservicio - lag(mextraccion_autoservicio, 1) over ventana_5 as mextraccion_autoservicio_delta_1_lag, mextraccion_autoservicio - lag(mextraccion_autoservicio, 2) over ventana_5 as mextraccion_autoservicio_delta_2_lag, mextraccion_autoservicio - lag(mextraccion_autoservicio, 3) over ventana_5 as mextraccion_autoservicio_delta_3_lag, mextraccion_autoservicio - lag(mextraccion_autoservicio, 4) over ventana_5 as mextraccion_autoservicio_delta_4_lag, mextraccion_autoservicio - lag(mextraccion_autoservicio, 5) over ventana_5 as mextraccion_autoservicio_delta_5_lag, avg(mextraccion_autoservicio) over ventana_5 as mextraccion_autoservicio_media_5
, regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_5 as mextraccion_autoservicio_slope_5, max(mextraccion_autoservicio) over ventana_5 as mextraccion_autoservicio_max_5, min(mextraccion_autoservicio) over ventana_5 as mextraccion_autoservicio_min_5, lag(cproductos, 1) over ventana_5 as cproductos_lag_1, lag(cproductos, 2) over ventana_5 as cproductos_lag_2, lag(cproductos, 3) over ventana_5 as cproductos_lag_3, lag(cproductos, 4) over ventana_5 as cproductos_lag_4, lag(cproductos, 5) over ventana_5 as cproductos_lag_5, cproductos - lag(cproductos, 1) over ventana_5 as cproductos_delta_1_lag, cproductos - lag(cproductos, 2) over ventana_5 as cproductos_delta_2_lag, cproductos - lag(cproductos, 3) over ventana_5 as cproductos_delta_3_lag, cproductos - lag(cproductos, 4) over ventana_5 as cproductos_delta_4_lag, cproductos - lag(cproductos, 5) over ventana_5 as cproductos_delta_5_lag, avg(cproductos) over ventana_5 as cproductos_media_5
, regr_slope(cproductos, cliente_antiguedad) over ventana_5 as cproductos_slope_5, max(cproductos) over ventana_5 as cproductos_max_5, min(cproductos) over ventana_5 as cproductos_min_5, lag(matm, 1) over ventana_5 as matm_lag_1, lag(matm, 2) over ventana_5 as matm_lag_2, lag(matm, 3) over ventana_5 as matm_lag_3, lag(matm, 4) over ventana_5 as matm_lag_4, lag(matm, 5) over ventana_5 as matm_lag_5, matm - lag(matm, 1) over ventana_5 as matm_delta_1_lag, matm - lag(matm, 2) over ventana_5 as matm_delta_2_lag, matm - lag(matm, 3) over ventana_5 as matm_delta_3_lag, matm - lag(matm, 4) over ventana_5 as matm_delta_4_lag, matm - lag(matm, 5) over ventana_5 as matm_delta_5_lag, avg(matm) over ventana_5 as matm_media_5
, regr_slope(matm, cliente_antiguedad) over ventana_5 as matm_slope_5, max(matm) over ventana_5 as matm_max_5, min(matm) over ventana_5 as matm_min_5
FROM competencia_03
WINDOW ventana_5 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 5 PRECEDING AND CURRENT ROW)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 12min 18s, sys: 1min 19s, total: 13min 38s
Wall time: 8min 31s


,Success


In [8]:
%%sql
select * from competencia_03 limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,matm_lag_5,matm_delta_1_lag,matm_delta_2_lag,matm_delta_3_lag,matm_delta_4_lag,matm_delta_5_lag,matm_media_5,matm_slope_5,matm_max_5,matm_min_5
0,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,NaN,NaN,NaN,NaN,NaN,NaN,30498.00,NaN,30498.0,30498.0
1,29185646,201902,1,0,1,58,232,21972.14,181302.16,3605.27,...,NaN,-30498.0,NaN,NaN,NaN,NaN,15249.00,-30498.000000,30498.0,0.0
2,29185646,201903,1,0,1,58,233,25201.17,199955.24,3311.49,...,NaN,11730.0,-18768.0,NaN,NaN,NaN,14076.00,-9384.000000,30498.0,0.0
3,29185646,201904,1,0,1,58,234,26398.48,219700.62,4821.07,...,NaN,-5865.0,5865.0,-24633.0,NaN,NaN,12023.25,-6216.900000,30498.0,0.0
4,29185646,201905,1,0,1,58,235,NaN,NaN,NaN,...,NaN,37536.0,31671.0,43401.0,12903.0,NaN,18298.80,3167.100000,43401.0,0.0
5,29185646,201906,1,0,1,58,236,30073.83,261052.72,2858.27,...,30498.0,-14076.0,23460.0,17595.0,29325.0,-1173.0,20136.50,3384.942857,43401.0,0.0
6,29185646,201907,1,0,1,58,237,29960.81,274608.22,4579.49,...,0.0,5865.0,-8211.0,29325.0,23460.0,35190.0,20918.50,7607.742857,43401.0,0.0
7,29185646,201908,1,0,1,58,238,29612.24,285747.10,8926.01,...,11730.0,5865.0,11730.0,-2346.0,35190.0,29325.0,27761.00,6300.685714,43401.0,5865.0
8,29185646,201909,1,0,1,58,239,29303.14,302594.30,5562.08,...,5865.0,-8797.5,-2932.5,2932.5,-11143.5,26392.5,31182.25,3736.842857,43401.0,5865.0
9,29185646,201910,1,0,1,58,240,NaN,NaN,NaN,...,43401.0,-14662.5,-23460.0,-17595.0,-11730.0,-25806.0,33137.25,-3267.642857,43401.0,17595.0


In [9]:
%%sql
copy competencia_03 to 'competencia_03_FEH_NA.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
